<html>
    <head>
        <h3>Wikipedia Data Transformer</h3>
    </head>
    <body>
        <ul>
          <li>Read XML dump files</li>
          <li>Extract: 'rev_id', 'timestamp', 'user_id', 'user_name', 'page_id', 'page_ns'</li>
          <li>Save the data to HDF files</li>
        </ul>
    </body>
</html>

In [1]:
import os
import gzip
import pandas as pd
from dateutil import parser

In [2]:
def get_value(text, tag):
    return text[len('<{}>'.format(tag)) : text.find('</{}>'.format(tag))]

In [17]:
def process_file(file_name, new_file_name):
    seg = 0
    data = []
    with gzip.open(file_name, 'rb') as f:  
        is_page = False
        is_revision = False
        is_contributor = False
        for i, line in enumerate(f):
            line = str(line)[1:].replace('\'', '').replace('\\n', '').strip()
            
            if line.startswith('<page>'):
                is_page = True
            elif line.startswith('</page>'):
                is_page = False
            elif line.startswith('<contributor>'):
                is_contributor = True
            elif line.startswith('</contributor>'):
                is_contributor = False
            elif line.startswith('<revision>'):
                is_revision = True
            elif line.startswith('</revision>'):
                is_revision = False
                data.append([rev_id, timestamp, user_id, user_name, page_id, page_ns]) 
            else:
                if is_page:
                    if line.startswith('<id') and not is_revision and not is_contributor:
                        page_id = int(get_value(line, 'id'))
                    if line.startswith('<ns'):
                        page_ns = int(get_value(line, 'ns'))

                if is_revision and not is_contributor:
                    if line.startswith('<id'):
                        rev_id = int(get_value(line, 'id'))
                    if line.startswith('<timestamp'):
                        timestamp = parser.parse(str(get_value(line, 'timestamp')))
                  
                if is_contributor:
                    if line.startswith('<ip'):
                        user_id = 0
                        user_name = str(get_value(line, 'ip'))
                    if line.startswith('<id'):
                        user_id = int(get_value(line, 'id'))
                    if line.startswith('<username'):
                        user_name = str(get_value(line, 'username'))

                        
            if len(data) > 0 and len(data) % 10000000 == 0:
                seg += 1          
                df = pd.DataFrame(data=data, columns=['rev_id', 'timestamp', 'user_id', 'user_name', 'page_id', 'page_ns'])
                df.to_hdf(new_file_name.replace('.h5', f'_p{seg}.h5'), key='enwiki')
                data = []
                del df
                
    df = pd.DataFrame(data=data, columns=['rev_id', 'timestamp', 'user_id', 'user_name', 'page_id', 'page_ns'])
    del data
    df.to_hdf(new_file_name, key='enwiki')
#     print(df.head(100))
    del df

In [42]:
def analyze_file(file_name):
    total_rows = 0
    with gzip.open(file_name, 'rb') as f:
        for i, line in enumerate(f):
            if i % 1000000 == 0 and i > 0:
                print(f"{i/1000000}M")
            total_rows = i
    print(f"Total rows: {total_rows}")

In [44]:
# base_path = r'\\asustor\download\enwiki\20180820\xml'
base_path = r'D:\data\enwiki\latest'
for root, dirs, files in os.walk(base_path):
    for file in files:
        file_name=os.path.join(root, file)
        new_file_name = file_name.replace('.xml.gz', '.h5')
        if os.path.exists(new_file_name):
            print('Exists: {}'.format(new_file_name))
            continue
        
        print('{} -> {}'.format(file_name, new_file_name))
#         analyze_file(file_name)
        process_file(file_name=file_name, new_file_name=new_file_name)
#         break

D:\data\enwiki\latest\enwiki-latest-stub-meta-history1.xml.gz -> D:\data\enwiki\latest\enwiki-latest-stub-meta-history1.h5
D:\data\enwiki\latest\enwiki-latest-stub-meta-history10.xml.gz -> D:\data\enwiki\latest\enwiki-latest-stub-meta-history10.h5
D:\data\enwiki\latest\enwiki-latest-stub-meta-history11.xml.gz -> D:\data\enwiki\latest\enwiki-latest-stub-meta-history11.h5
D:\data\enwiki\latest\enwiki-latest-stub-meta-history12.xml.gz -> D:\data\enwiki\latest\enwiki-latest-stub-meta-history12.h5
D:\data\enwiki\latest\enwiki-latest-stub-meta-history13.xml.gz -> D:\data\enwiki\latest\enwiki-latest-stub-meta-history13.h5
D:\data\enwiki\latest\enwiki-latest-stub-meta-history14.xml.gz -> D:\data\enwiki\latest\enwiki-latest-stub-meta-history14.h5
D:\data\enwiki\latest\enwiki-latest-stub-meta-history15.xml.gz -> D:\data\enwiki\latest\enwiki-latest-stub-meta-history15.h5
D:\data\enwiki\latest\enwiki-latest-stub-meta-history16.xml.gz -> D:\data\enwiki\latest\enwiki-latest-stub-meta-history16.h5
D: